In [2]:
import torch
from torch import Tensor
from torch import nn
from torch import optim
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader, random_split
from PIL import Image
import torchvision as tv
from torchvision import models
import os
import numpy
from tqdm import tqdm

import zipfile
from google.colab import drive

drive.mount('/content/drive')
zipped_test = '/content/drive/MyDrive/Colab Notebooks/dogs_muffins/test.zip'
zipped_train = '/content/drive/MyDrive/Colab Notebooks/dogs_muffins/train.zip'
z = zipfile.ZipFile(zipped_test, 'r')
z.extractall(path='/content/dataset')
z = zipfile.ZipFile(zipped_train, 'r')
z.extractall(path='/content/dataset')
import gc
gc.collect()


DATASET_ROOT_DIR = '/content/dataset/'
BATCH_SIZE = 64
if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'
device


transforms = tv.transforms.Compose([
    tv.transforms.Resize((224, 224)),
    tv.transforms.ToTensor(),
    tv.transforms.RandomHorizontalFlip(0.5),
])


dataset = ImageFolder(root=os.path.join(DATASET_ROOT_DIR, 'train'),
                      transform=transforms
                      ) # 0 - dogs


train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])


train_data = DataLoader(
                  train_dataset,
                  batch_size=BATCH_SIZE,
                  shuffle=True,
                  )
test_data = DataLoader(
                  test_dataset,
                  batch_size=BATCH_SIZE,
                  shuffle=True,
                  )


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
class CustomResNet(nn.Module):
    def __init__(self):
        super(CustomResNet, self).__init__()
        self.resnet = models.resnet18(pretrained=False)

        self.ep = 0

        num_ftrs = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_ftrs, 2)

    def forward(self, x):
        return self.resnet(x)


class CustomResNetXt50_32x4d(nn.Module):
    def __init__(self):
        super().__init__()
        self.ep = 0

        self.main = models.resnext50_32x4d(pretrained=False)

        num_ftrs = self.main.fc.in_features
        self.main.fc = nn.Linear(num_ftrs, 2)

    def forward(self, x):
        return self.main(x)

model = CustomResNetXt50_32x4d()
model.to(device)


model = CustomResNet()
model.to(device)


actual_ep=175
model = torch.load(f'/content/drive/MyDrive/Colab Notebooks/models/resnet_model_x50_{actual_ep}')


loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-4)


def save_model_to_drive(model, name):
  pth = os.path.join('/content/drive/MyDrive/Colab Notebooks/models', name)
  torch.save(model, pth)


def get_acc():
  correct = 0
  total = 0
  model.eval()
  with torch.no_grad():
      for img, label in test_data:
          img = img.to(device)
          label = label.to(device)
          out = model(img)
          _, predicted = torch.max(out.data, 1)
          total += label.size(0)
          correct += (predicted == label).sum().item()

  return correct / total



EPOCHS = 1


for i in range(EPOCHS):
  model.train()
  for img, label in tqdm(train_data):
    optimizer.zero_grad()
    img = img.to(device)
    label = label.to(device)
    out = model(img)
    loss = loss_func(out, label)
    loss.backward()
    optimizer.step()
  model.ep += 1

  print(f'on epoch {model.ep} loss: {loss.item()}')
  if model.ep % 5 == 0:
    print(f'accuracy: {get_acc()}')
    save_model_to_drive(model, f'resnet_model_x50_{model.ep}')
    print('model saved!')

get_acc()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
100%|██████████| 60/60 [00:51<00:00,  1.17it/s]


on epoch 176 loss: 0.39921391010284424


0.9809926082365364

In [4]:

x = sum(p.numel() for p in model.parameters())
print(f'{x:,}')

m = tv.models.resnext50_32x4d()
x = sum(p.numel() for p in m.parameters())
print(f'{x:,}')

from datetime import datetime
device = 'cuda'
model.to(device)
a = datetime.now()
im = train_dataset[3000][0].unsqueeze(0)
im = im.to(device)
print(im.shape)

out = model(im)
b = datetime.now()
print(b-a)

class MyDataset(Dataset):
  def __init__(self, root, transforms=None):
    self.images = [f for f in os.listdir(root)]
    self.root = root
    self.transforms = transforms

  def __len__(self):
    return len(self.images)

  def __getitem__(self, idx):
    img_name = os.path.join(self.root, self.images[idx])
    image = Image.open(img_name)
    if self.transforms:
      image = self.transforms(image)
    return image, img_name

transforms_ = tv.transforms.Compose([
    tv.transforms.Resize((224, 224)),
    tv.transforms.ToTensor(),
])

d = MyDataset('/content/dataset/test', transforms_)

dtldr = DataLoader(
                  d,
                  batch_size=1,
                  shuffle=True,
                  )


for i, (img, img_name) in enumerate(dtldr):
  img = img.to(device)
  out = model(img)
  if torch.argmax(out).item() == 0:
    name = 'dog'
  else:
    name = 'muf'

  new = os.path.join('/content/dataset/test', name+str(i))
  os.rename(img_name[0], new)



11,177,538
25,028,904
torch.Size([1, 3, 224, 224])
0:00:00.033767


In [21]:
import os
from PIL import Image
import pandas as pd
import zipfile

test_zip_path = '/content/drive/MyDrive/Colab Notebooks/dogs_muffins/test.zip'
new_extracted_folder = '/content/dataset/new_test/'


with zipfile.ZipFile(test_zip_path, 'r') as zip_ref:
    zip_ref.extractall(new_extracted_folder)

extracted_folder = '/content/dataset/new_test/test/'


predictions = []

model.eval()

for image_file in os.listdir(extracted_folder):
    image_path = os.path.join(extracted_folder, image_file)


    if os.path.isfile(image_path):

        with Image.open(image_path) as img:

            img = transforms(img).unsqueeze(0).to(device)


            with torch.no_grad():
                model.eval()
                output = model(img)
                _, predicted = torch.max(output, 1)


            predictions.append({'ID': image_file, 'Label': predicted.item()})
            #0=dogs  1=muffins


predictions_df = pd.DataFrame(predictions)


predictions_df.to_csv('/content/drive/MyDrive/Colab Notebooks/predictions.csv', index=False)


KeyboardInterrupt: ignored

# Новый раздел